In [168]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#plt.rcParams['figure.figsize'] = (14, 6)

In [169]:
def to_col(x):   
    if len(x.shape)==1:
        x=x.reshape(x.shape[0],1)
    return x

def to_line(x):
    """ convert an vector to line vector if needed """
    if len(x.shape)==1:
        x=x.reshape(1,x.shape[0])
    return x

def plot_data(data,labels=None):
    """
    Affiche des donnees 2D
    :param data: matrice des donnees 2d
    :param labels: vecteur des labels (discrets)
    :return:
    """
    cols,marks = ["red", "green", "blue", "orange", "black", "cyan"],[".","+","*","o","x","^"]
    if labels is None:
        plt.scatter(data[:,0],data[:,1],marker="x")
        return
    for i,l in enumerate(sorted(list(set(labels.flatten())))):
        plt.scatter(data[labels==l,0],data[labels==l,1],c=cols[i],marker=marks[i])
        

def make_grid(data=None,xmin=-5,xmax=5,ymin=-5,ymax=5,step=20):
    """ Cree une grille sous forme de matrice 2d de la liste des points
    :param data: pour calcluler les bornes du graphe
    :param xmin: si pas data, alors bornes du graphe
    :param xmax:
    :param ymin:
    :param ymax:
    :param step: pas de la grille
    :return: une matrice 2d contenant les points de la grille
    """
    if data is not None:
        xmax, xmin, ymax, ymin = np.max(data[:,0]),  np.min(data[:,0]), np.max(data[:,1]), np.min(data[:,1])
    x, y =np.meshgrid(np.arange(xmin,xmax,(xmax-xmin)*1./step), np.arange(ymin,ymax,(ymax-ymin)*1./step))
    grid=np.c_[x.ravel(),y.ravel()]
    return grid, x, y


def plot_frontiere(data,f,step=20):
    """ Trace un graphe de la frontiere de decision de f
    :param data: donnees
    :param f: fonction de decision
    :param step: pas de la grille
    :return:
    """
    grid,x,y=make_grid(data=data,step=step)
    plt.contourf(x,y,f(grid).reshape(x.shape),colors=('gray','blue'),levels=[-1,0,1])

## Chargement des données

In [225]:
A_train = np.loadtxt('classification_data_HWK1/classificationA.train')
B_train = np.loadtxt('classification_data_HWK1/classificationB.train')
C_train = np.loadtxt('classification_data_HWK1/classificationC.train')

A_test = np.loadtxt('classification_data_HWK1/classificationA.test')
B_test = np.loadtxt('classification_data_HWK1/classificationB.test')
C_test = np.loadtxt('classification_data_HWK1/classificationC.test')

In [227]:
X_A_train = A_train[:,0:2]
Y_A_train = to_col(A_train[:,2])
#X_A_train,Y_A_train

X_B_train = B_train[:,0:2]
Y_B_train = to_col(B_train[:,2])
#X_B_train,Y_B_train

X_C_train = C_train[:,0:2]
Y_C_train = to_col(C_train[:,2])
#X_C_train,Y_C_train

X_A_test = A_test[:,0:2]
Y_A_test = to_col(A_test[:,2])
#X_A_test,Y_A_test

X_B_test = B_test[:,0:2]
Y_B_test = to_col(B_test[:,2])
#X_B_test,Y_B_test

X_C_test = C_test[:,0:2]
Y_C_test = to_col(C_test[:,2])
#X_C_test,Y_C_test

plt.rcParams['figure.figsize'] = (14, 6)

fig, axes = plt.subplots(2, 3)
green, = axes[0][0].plot([a[0] for a in A_train if a[2]==1],[a[1] for a in A_train if a[2]==1],'go')
blue, = axes[0][0].plot([a[0] for a in A_train if a[2]==0],[a[1] for a in A_train if a[2]==0],'bo')

axes[0][1].plot([b[0] for b in B_train if b[2]==1],[b[1] for b in B_train if b[2]==1],'go')
axes[0][1].plot([b[0] for b in B_train if b[2]==0],[b[1] for b in B_train if b[2]==0],'bo')

axes[0][2].plot([c[0] for c in C_train if c[2]==1],[c[1] for c in C_train if c[2]==1],'go')
axes[0][2].plot([c[0] for c in C_train if c[2]==0],[c[1] for c in C_train if c[2]==0],'bo')

axes[1][0].plot([a[0] for a in A_test if a[2]==1],[a[1] for a in A_test if a[2]==1],'go')
axes[1][0].plot([a[0] for a in A_test if a[2]==0],[a[1] for a in A_test if a[2]==0],'bo')

axes[1][1].plot([b[0] for b in B_test if b[2]==1],[b[1] for b in B_test if b[2]==1],'go')
axes[1][1].plot([b[0] for b in B_test if b[2]==0],[b[1] for b in B_test if b[2]==0],'bo')

axes[1][2].plot([c[0] for c in C_test if c[2]==1],[c[1] for c in C_test if c[2]==1],'go')
axes[1][2].plot([c[0] for c in C_test if c[2]==0],[c[1] for c in C_test if c[2]==0],'bo')

axes[0][0].set_title("A",fontsize="40")
axes[0][1].set_title("B",fontsize="40")
axes[0][2].set_title("C",fontsize="40")
axes[0][0].set_ylabel("Train",fontsize="40")
axes[1][0].set_ylabel("Test",fontsize="40")
fig.legend((green,blue),('1', '0'),'center right',numpoints = 1)

plt.show()

## 2. MLE Generative model (LDA)

In [228]:
def theta_MLE(Y):
    return(np.sum(Y)/Y.shape[0])

def mu_1_MLE(X,Y):
    return([np.sum(Y*to_col(X[:,0]))/np.sum(Y), np.sum(Y*to_col(X[:,1]))/np.sum(Y)])

def mu_0_MLE(X,Y): 
    return([(np.sum(X[:,0]) - np.sum(Y*to_col(X[:,0])))/(Y.shape[0]-np.sum(Y)), 
            (np.sum(X[:,1]) - np.sum(Y*to_col(X[:,1])))/(Y.shape[0]-np.sum(Y))])

def sigma_MLE(X,Y): 
    s1 = np.dot(np.transpose(Y)*np.transpose(X-mu_1_MLE(X,Y)),(X-mu_1_MLE(X,Y)))
    s0 = np.dot((np.ones(Y.shape[0])-np.transpose(Y))*np.transpose(X-mu_0_MLE(X,Y)),(X-mu_0_MLE(X,Y)))
    return((s1+s0)/Y.shape[0])

def sigmoid(X):  #fonction sigmoide pour la proba conditionnelle
    return (1/(1+np.exp(-X)))

def a_sigmoid(X,Y):
    return(np.dot(np.linalg.inv(sigma_MLE(X,Y)),np.asarray(mu_1_MLE(X,Y))-np.asarray(mu_0_MLE(X,Y))))

def b_sigmoid(X,Y):
    return(np.log(theta_MLE(Y)/(1-theta_MLE(Y))) 
           -1/2*np.dot(np.dot((np.asarray(mu_1_MLE(X,Y))-np.asarray(mu_0_MLE(X,Y))),np.linalg.inv(sigma_MLE(X,Y)))
                ,(np.asarray(mu_0_MLE(X,Y))+np.asarray(mu_1_MLE(X,Y)))))

def LDA(x):
    return(sigmoid(np.transpose(a_sigmoid)*x+b_sigmoid))

#### Sample A

In [229]:
print("theta =",theta_MLE(Y_A_train))
print("mu_1 =",mu_1_MLE(X_A_train,Y_A_train))
print("mu_0 =",mu_0_MLE(X_A_train,Y_A_train))
print("sigma =",sigma_MLE(X_A_train,Y_A_train))
print("a =", a_sigmoid(X_A_train,Y_A_train))
print("b =", b_sigmoid(X_A_train,Y_A_train))

theta = 0.333333333333
mu_1 = [-2.6923200424000004, 0.86604199999999987]
mu_0 = [2.8997094650999999, -0.89387399999999984]
sigma = [[ 2.44190897 -1.13194024]
 [-1.13194024  0.61375465]]
a = [-6.62245258 -9.3462503 ]
b = -0.136496290948


In [232]:
a_sigmoid=a_sigmoid(X_A_test,Y_A_test)
b_sigmoid=b_sigmoid(X_A_test,Y_A_test)
LDA(X_A_test)

TypeError: 'numpy.ndarray' object is not callable

In [231]:
plt.figure()
#y = Y_test_A.reshape(Y_test_A.shape[0],)
plot_frontiere(X_A_test,LDA,50)
plot_data(X_A_test,Y_A_test)
#plt.title("Classification of test set A by LDA")
plt.show()

ValueError: total size of new array must be unchanged

In [212]:
sigmoid(np.dot(X_A_train,a_sigmoid(X_A_train,Y_A_train))+b_sigmoid(X_A_train,Y_A_train))
np.dot(X_A_train,a_sigmoid(X_A_train,Y_A_train))
sigmoid(X_A_train*a_sigmoid(X_A_train,Y_A_train)+b_sigmoid(X_A_train,Y_A_train))

array([[  1.00000000e+00,   3.59762903e-06],
       [  1.00000000e+00,   5.77810165e-09],
       [  9.99949872e-01,   7.65460108e-03],
       [  9.99504855e-01,   2.40778432e-02],
       [  2.66403973e-02,   9.97541787e-01],
       [  6.90438999e-01,   9.98473809e-01],
       [  1.00000000e+00,   4.12361314e-06],
       [  9.99986101e-01,   9.74119058e-02],
       [  9.99998677e-01,   2.21426182e-03],
       [  9.99999988e-01,   2.07721762e-05],
       [  1.00000000e+00,   5.25298512e-06],
       [  9.99998596e-01,   1.08140572e-03],
       [  1.00000000e+00,   1.11986105e-08],
       [  9.99999989e-01,   4.23002381e-04],
       [  1.00000000e+00,   3.14850186e-05],
       [  5.04787657e-01,   8.24445773e-01],
       [  1.00000000e+00,   1.33666754e-07],
       [  1.00000000e+00,   1.01960077e-09],
       [  1.00000000e+00,   3.15933823e-06],
       [  1.00000000e+00,   3.43225009e-07],
       [  9.99999987e-01,   1.42025016e-03],
       [  1.00000000e+00,   1.92071521e-13],
       [  

#### Sample B

In [184]:
print("theta =",theta_MLE(Y_B_train))
print("mu_1 =",mu_1_MLE(X_B_train,Y_B_train))
print("mu_0 =",mu_0_MLE(X_B_train,Y_B_train))
print("sigma =",sigma_MLE(X_B_train,Y_B_train))
print("a =", a_sigmoid(X_B_train,Y_B_train))
print("b =", b_sigmoid(X_B_train,Y_B_train))

theta = 0.5
mu_1 = [-3.2167073426666666, 1.0830673333333334]
mu_0 = [3.340688964066667, -0.83546333333333334]
sigma = [[ 3.34623467 -0.13516489]
 [-0.13516489  1.73807475]]
a = [-1.92108197  0.95442836]
b = 0.000929288716542


#### Sample C

In [185]:
print("theta =",theta_MLE(Y_C_train))
print("mu_1 =",mu_1_MLE(X_C_train,Y_C_train))
print("mu_0 =",mu_0_MLE(X_C_train,Y_C_train))
print("sigma =",sigma_MLE(X_C_train,Y_C_train))
print("a =", a_sigmoid(X_C_train,Y_C_train))
print("b =", b_sigmoid(X_C_train,Y_C_train))

theta = 0.625
mu_1 = [-2.9423288508400001, -0.95782840000000014]
mu_0 = [2.7930482376000003, -0.8383866666666665]
sigma = [[ 2.88039225 -0.63405081]
 [-0.63405081  5.19952435]]
a = [-2.05129911 -0.27311529]
b = 0.112429132177


## 5. QDA model

In [ ]:
def sigma_0_MLE(X,Y):
    s0 =np.dot((np.ones(Y.shape[0])-np.transpose(Y))*np.transpose(X-mu_0_MLE(X,Y)),(X-mu_0_MLE(X,Y)))
    return(s0/(Y.shape[0]-np.sum(Y)))

def sigma_1_MLE(X,Y):
    s1 = np.dot(np.transpose(Y)*np.transpose(X-mu_1_MLE(X,Y)),(X-mu_1_MLE(X,Y)))
    return(s1/np.sum(Y))







def c_sigmoid(X,Y):
    return(np.dot(np.linalg.inv(sigma_1_MLE(X,Y)),np.asarray(nu_1_MLE(X,Y)))
           -np.dot(np.linalg.inv(sigma_0_MLE(X,Y)),np.asarray(nu_0_MLE(X,Y)))
           
           
           
           np.asarray(nu_0_MLE(X,Y))))

def d_sigmoid(X,Y):
    return(np.log(theta_MLE(Y)/(1-theta_MLE(Y))) 
           -1/2*np.dot(np.dot((np.asarray(nu_1_MLE(X,Y))-np.asarray(nu_0_MLE(X,Y))),np.linalg.inv(sigma_MLE(X,Y)))
                ,(np.asarray(nu_0_MLE(X,Y))+np.asarray(nu_1_MLE(X,Y)))))

def M_sigmoid(X,Y):
    return(np.dot(np.linalg.inv(sigma_MLE(X,Y)),np.asarray(nu_1_MLE(X,Y))-np.asarray(nu_0_MLE(X,Y))))



In [122]:
#LDA(X_A_train, Y_A_train)

In [ ]:
Theta_MLE_A = (1.0/np.shape(A_train)[0])*sum([a[2] for a in A_train])
Theta_MLE_B = (1.0/np.shape(B_train)[0])*sum([b[2] for b in B_train])
Theta_MLE_C = (1.0/np.shape(C_train)[0])*sum([c[2] for c in C_train])

print Theta_MLE_A
print Theta_MLE_B
print Theta_MLE_C

mu_0_MLE_A = np.sum([(1-a[2])*np.array([a[0],a[1]]) for a in A_train],axis = 0)/sum([(1-a[2]) for a in A_train])
mu_0_MLE_B = np.sum([(1-b[2])*np.array([b[0],b[1]]) for b in B_train],axis = 0)/sum([(1-b[2]) for b in B_train])
mu_0_MLE_C = np.sum([(1-c[2])*np.array([c[0],c[1]]) for c in C_train],axis = 0)/sum([(1-c[2]) for c in C_train])

print mu_0_MLE_A
print mu_0_MLE_B
print mu_0_MLE_C

mu_1_MLE_A = np.sum([a[2]*np.array([a[0],a[1]]) for a in A_train],axis = 0)/sum([a[2] for a in A_train])
mu_1_MLE_B = np.sum([b[2]*np.array([b[0],b[1]]) for b in B_train],axis = 0)/sum([b[2] for b in B_train])
mu_1_MLE_C = np.sum([c[2]*np.array([c[0],c[1]]) for c in C_train],axis = 0)/sum([c[2] for c in C_train])

print mu_1_MLE_A
print mu_1_MLE_B
print mu_1_MLE_C

def p_Y1_X(x,theta,mu_0,mu_1,sigma):#x vecteur vertical
    return( 1.0/(1.0+((1.0-theta)/(theta))*exp((((mu_0+mu_1)[np.newaxis]).T).dot(sigma).dot(x) - (1.0/2)*((mu_0.T).dot(sigma).dot(mu_0) + (mu_1.T).dot(sigma).dot(mu_1))))
    

In [ ]:
print A_train[:3]

In [ ]:
print [np.array([a[0],a[1]]) for a in A_train][:2]

In [ ]:
a = np.array([1,2])
b = np.array([[3,4],[5,6]])
print np.transpose(a[np.newaxis])*b

In [ ]:
#transformer un vecteur ligne en colone
vecteur_ligne = np.array([1,2,3])
vecteur_colone = np.transpose(vecteur_ligne[np.newaxis])
print vecteur_ligne
print np.transpose(vecteur_colone).dot(np.array([[1,2,0],[0,1,0],[0,0,1]])).dot(vecteur_colone)

print vecteur_colone.T

In [ ]:
#logistic regression
